In [ ]:
import sys
from pathlib import Path
import torch
import requests
import json
import os 

model_name="large-v2"
pattern = "n_best_phoneme"



from dataclasses import dataclass, field, replace
from typing import TYPE_CHECKING, Dict, Iterable, List, Optional, Sequence, Tuple, Union
import whisper_openAI.whisper as whisper
import torch
from whisper_openAI.whisper.tokenizer import Tokenizer, get_tokenizer
import torch
import torch.nn.functional as F
from torch import Tensor
import json

# We get the acoustic embeddings from Whisper Large V2
model,processor = whisper.load_model("large-v2")
# model,processor = whisper.load_model("medium")

In [2]:
speaker_id = 'M03'
test_speaker = speaker_id

In [ ]:
with open(f'Inference/gs_inferences/large-v2_hypo/torgo_train_{speaker_id}_{model_name}.json', "r") as file:  # Change the file path and name here
    train_data = json.load(file)

with open(f'Inference/gs_inferences/large-v2_hypo/torgo_val_{speaker_id}_{model_name}.json', "r") as valid_file:
    val_data = json.load(valid_file)

# Load the test set
with open(f'Inference/gs_inferences/large-v2_hypo/torgo_test_{speaker_id}_{model_name}.json', "r") as test_file:
    test_data = json.load(test_file)
    

from lit_llama.tokenizer import Tokenizer
from tqdm import tqdm


tokenizer_path: Path = Path("weights/tokenizer.model")
tokenizer = Tokenizer(tokenizer_path)
print(f"train has {len(train_data):,} samples")

train has 9,376 samples


In [4]:
import torch
old_folder = "tiny_hypo"
old_data_train = torch.load(f'Inference/gs_inferences/{old_folder}/torgo_{test_speaker}_train.pt',map_location=torch.device('cpu'))
old_data_val = torch.load(f'Inference/gs_inferences/{old_folder}/torgo_{test_speaker}_val.pt',map_location=torch.device('cpu'))
old_data_test = torch.load(f'Inference/gs_inferences/{old_folder}/torgo_{test_speaker}_test.pt',map_location=torch.device('cpu'))


# In[8]:


# Print the lengths of the loaded data
print(f'Length of old_data_train: {len(old_data_train)}')
print(f'Length of old_data_val: {len(old_data_val)}')
print(f'Length of old_data_test: {len(old_data_test)}')

Length of old_data_train: 9376
Length of old_data_val: 460
Length of old_data_test: 442


In [5]:
train_data[0]

{'F01-Session1-arrayMic-0009': {'temp': 0.74,
  'path': '/work/van-speech-nlp/data/torgo/F01/Session1/wav_arrayMic/0009.wav',
  'ground_truth': 'pat',
  'inference': ['',
   'ah',
   'ah!',
   'alright',
   'bartender 3',
   'berk',
   'bleep',
   'blurp',
   'burp',
   'burp!',
   'burps',
   'but',
   'bye!',
   'ehh',
   'ha',
   'hah',
   'hehe',
   'hot!',
   'hrrrgh',
   'huh',
   'hurt!',
   'huuuuh!',
   'im gonna go',
   'oh',
   'purr!',
   'rip camera man',
   'thanks for watching see you next time!',
   'ugh',
   'uh',
   'uhoh',
   'urgh!'],
  'source': 'NP-Torgo',
  'category': 'NP-Torgo',
  'time': 30.0}}

In [24]:
from g2p_en import G2p
import re

# Define the convert_to_phonemes function
def convert_to_phonemes(sentences, remove_num=True):
    g2p = G2p()
    phoneme_sequences = []

    for sentence in sentences:
        phonemes = " ".join(g2p(sentence))
        if remove_num:
            phonemes = re.sub(r'\d+', '', phonemes)
        phoneme_sequences.append(phonemes)

    return phoneme_sequences

def tokenize(tokenizer: Tokenizer, string: str, max_length: int, eos=True) -> torch.Tensor:
    return tokenizer.encode(string, bos=True, eos=eos, max_length=max_length)
    
# Define the process_train_data function
def process_train_data(train_data, old_data):
    instruction = 'You are an ASR transcript selector. You have a few transcripts generated by an automatic speech recognition model, and their corresponding phonemes. Your task is to generate the most likely transcript from them. If the generated transcripts have grammatical or logical errors, you will modify them accordingly to produce the most accurate and coherent transcript.'
    result = []

    for i in tqdm(range(len(train_data[:2]))):
        for name in train_data[i].keys():
            ip = train_data[i][name]
        
        inference = ip['inference']
        gt = ip['ground_truth']
        path = ip['path']
        
        # Removing the ground_truth, if present among the inferences for the prompt
        if gt in inference:
            inference.remove(gt)

        print(gt)
        print(inference)  # Print the number of inferences for each prompt
        
        # Convert inference to phoneme sequences
        phoneme_sequences = convert_to_phonemes(inference[:15])
        print(phoneme_sequences)
                                                                                                                          b6
        # Joining the inputs with '\n'
        for_input = "Possible Transcripts: \n" + '\n'.join(inference[:15])
        for_input += "\n" + "Possible Phoenme Sequences: \n" + '\n'.join(phoneme_sequences[:15])
          
        print(f"{for_input} \n")
        # The prompt follows the Alpaca template
        full_prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{for_input}\n\n### Response:"""
        full_prompt_and_response = full_prompt + gt

        encoded_full_prompt = tokenize(tokenizer, full_prompt, max_length=2048, eos=False)
        encoded_full_prompt_and_response = tokenize(tokenizer, full_prompt_and_response, eos=True, max_length=2048)
        labels = encoded_full_prompt_and_response.clone()
        labels_with_masked_input = encoded_full_prompt_and_response.clone()
        labels_with_masked_input[:len(encoded_full_prompt)] = -1
        
        audio_features = old_data[i]["audio_features"]
        
        result.append({**ip, 'index': name, "input_ids": encoded_full_prompt_and_response, "input_ids_no_response": encoded_full_prompt, "labels": labels, 'labels_with_masked_input': labels_with_masked_input, 'audio_features': audio_features.bfloat16()})
    
    return result

In [25]:
split = "train"
result = process_train_data(train_data, old_data_train)
torch.save(result,f'Inference/gs_inferences/torgo_{speaker_id}_{model_name}_{pattern}_{split}.pt')
print(f"Processed {split} data and saved checkpoint for {speaker_id}")

  0%|          | 0/2 [00:00<?, ?it/s]

pat
['', 'ah', 'ah!', 'alright', 'bartender 3', 'berk', 'bleep', 'blurp', 'burp', 'burp!', 'burps', 'but', 'bye!', 'ehh', 'ha', 'hah', 'hehe', 'hot!', 'hrrrgh', 'huh', 'hurt!', 'huuuuh!', 'im gonna go', 'oh', 'purr!', 'rip camera man', 'thanks for watching see you next time!', 'ugh', 'uh', 'uhoh', 'urgh!']


 50%|█████     | 1/2 [00:01<00:01,  1.14s/it]

['', 'AA', 'AA   !', 'AO L R AY T', 'B AA R T EH N D ER   TH R IY', 'B ER K', 'B L IY P', 'B L ER P', 'B ER P', 'B ER P   !', 'B ER P S', 'B AH T', 'B AY   !', 'EH HH', 'HH AA']
Possible Transcripts: 

ah
ah!
alright
bartender 3
berk
bleep
blurp
burp
burp!
burps
but
bye!
ehh
ha
Possible Phoenme Sequences: 

AA
AA   !
AO L R AY T
B AA R T EH N D ER   TH R IY
B ER K
B L IY P
B L ER P
B ER P
B ER P   !
B ER P S
B AH T
B AY   !
EH HH
HH AA 

meat
['eat', 'great', 'heh heeh', 'hey', 'hey hey', 'lets not get ahead of ourselves', 'little asleep', 'neat', 'right', 'wait', 'wait wait', 'we', 'ねえ。 nee']


100%|██████████| 2/2 [00:02<00:00,  1.03s/it]

['IY T', 'G R EY T', 'HH EH   HH IY HH', 'HH EY', 'HH EY   HH EY', 'L EH T S   N AA T   G EH T   AH HH EH D   AH V   AW ER S EH L V Z', 'L IH T AH L   AH S L IY P', 'N IY T', 'R AY T', 'W EY T', 'W EY T   W EY T', 'W IY', 'N IY']
Possible Transcripts: 
eat
great
heh heeh
hey
hey hey
lets not get ahead of ourselves
little asleep
neat
right
wait
wait wait
we
ねえ。 nee
Possible Phoenme Sequences: 
IY T
G R EY T
HH EH   HH IY HH
HH EY
HH EY   HH EY
L EH T S   N AA T   G EH T   AH HH EH D   AH V   AW ER S EH L V Z
L IH T AH L   AH S L IY P
N IY T
R AY T
W EY T
W EY T   W EY T
W IY
N IY 

Processed train data and saved checkpoint for M03
